# ArizeDatasetEmbeddings Guard
In this demo, we are going to set up and use a Guard that blocks the LLM from responding from attempted jailbreaks. We will do this using the `ArizeDatasetEmbeddings` Guard from Arize AI. This Guard works in the following way:
*   The Guard computes embeddings for chunks associated with a set of few shot examples of "bad" user prompts or LLM messages (we recommend using 10 different prompts)
*   When the the Guard is applied to a user or LLM message, the Guard computes the embedding for the input message and checks if any of the few shot "train" examples in the dataset are close to the message in embedded space
*   If the cosine distance between the input message and any of the chunks is within the user-specified threshold (default setting is 0.2), then the Guard intercepts the LLM call.

In this demo, we use the `ArizeDatasetEmbeddings` Guard in two ways, first on a dataset of jailbreak prompts, then subsequently on a dataset of PII prompts. In both cases, we apply the Guard to user input messages, rather than LLM output messages (although we could take either approach). If the Guard flags a jailbreak attempt or PII in the user message, we simply throw an `Exception`. Alternatively, in practice the user can specify a default LLM response that can be used when the Guard is triggered.


# Install Dependencies
Various installations are required for OTL, LlamaIndex and Open AI.

In [14]:
!pip install -qq 'openinference-instrumentation-llama-index>=0.1.6' 'openinference-instrumentation-llama-index>=0.1.6'  llama-index-llms-openai opentelemetry-exporter-otlp llama-index>=0.10.3 "llama-index-callbacks-arize-phoenix>=0.1.2" arize-otel

import os
from getpass import getpass

openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


# Initialize Arize Phoenix
Set up OTL tracer for the `LlamaIndexInstrumentor`.

In [15]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from arize_otel import register_otel, Endpoints

# Setup OTEL via our convenience function
register_otel(
    endpoints = Endpoints.ARIZE,
    space_key = getpass("🔑 Enter your Arize space key in the space settings page of the Arize UI: "),
    api_key = getpass("🔑 Enter your Arize API key in the space settings page of the Arize UI: "),
    model_id = "test-guard-july10-6:07pm", # name this to whatever you would like
)
LlamaIndexInstrumentor().instrument()

🔑 Enter your Arize space key in the space settings page of the Arize UI: ··········
🔑 Enter your Arize API key in the space settings page of the Arize UI: ··········


Overriding of current TracerProvider is not allowed


Attempting to instrument while already instrumented


# Instrument Guardrails AI
Install and instrument Guardrails AI. Import `ArizeDatasetEmbeddings` Guard.

In [16]:
!pip install -qq guardrails-ai litellm

In [95]:
import os
from typing import Any, Callable, Dict, Optional, Union, Type
from warnings import warn
from enum import Enum
from abc import ABC, abstractmethod

from guardrails.validator_base import (
    FailResult,
    PassResult,
    ValidationResult,
    Validator,
    register_validator,
)
from guardrails.stores.context import get_call_kwarg
from litellm import completion, get_llm_provider


class ArizeRagEvalPromptBase(ABC):
    def __init__(self, prompt_name, **kwargs) -> None:
        self.prompt_name = prompt_name

    @abstractmethod
    def generate_prompt(self, user_input_message: str, reference_text: str, llm_response: str) -> str:
        pass


class ContextRelevancyPrompt(ArizeRagEvalPromptBase):
    def generate_prompt(self, user_input_message: str, reference_text: str, llm_response: str) -> str:
        return f"""
            You are comparing a reference text to a question and trying to determine if the reference text
            contains information relevant to answering the question. Here is the data:
                [BEGIN DATA]
                ************
                [Question]: {user_input_message}
                ************
                [Reference text]: {reference_text}
                ************
                [END DATA]
            Compare the Question above to the Reference text. You must determine whether the Reference text
            contains information that can answer the Question. Please focus on whether the very specific
            question can be answered by the information in the Reference text.
            Your response must be single word, either "relevant" or "unrelated",
            and should not contain any text or characters aside from that word.
            "unrelated" means that the reference text does not contain an answer to the Question.
            "relevant" means the reference text contains an answer to the Question."""


class HallucinationPrompt(ArizeRagEvalPromptBase):
    def generate_prompt(self, user_input_message: str, reference_text: str, llm_response: str) -> str:
        return f"""
            In this task, you will be presented with a query, a reference text and an answer. The answer is
            generated to the question based on the reference text. The answer may contain false information. You
            must use the reference text to determine if the answer to the question contains false information,
            if the answer is a hallucination of facts. Your objective is to determine whether the answer text
            contains factual information and is not a hallucination. A 'hallucination' refers to
            an answer that is not based on the reference text or assumes information that is not available in
            the reference text. Your response should be a single word: either "factual" or "hallucinated", and
            it should not include any other text or characters. "hallucinated" indicates that the answer
            provides factually inaccurate information to the query based on the reference text. "factual"
            indicates that the answer to the question is correct relative to the reference text, and does not
            contain made up information. Please read the query and reference text carefully before determining
            your response.

                [BEGIN DATA]
                ************
                [Query]: {user_input_message}
                ************
                [Reference text]: {reference_text}
                ************
                [Answer]: {llm_response}
                ************
                [END DATA]

                Is the answer above factual or hallucinated based on the query and reference text?
            """


class QACorrectnessPrompt(ArizeRagEvalPromptBase):
    def generate_prompt(self, user_input_message: str, reference_text: str, llm_response: str) -> str:
        return f"""
            You are given a question, an answer and reference text. You must determine whether the
            given answer correctly answers the question based on the reference text. Here is the data:
                [BEGIN DATA]
                ************
                [Question]: {user_input_message}
                ************
                [Reference]: {reference_text}
                ************
                [Answer]: {llm_response}
                [END DATA]
            Your response must be a single word, either "correct" or "incorrect",
            and should not contain any text or characters aside from that word.
            "correct" means that the question is correctly and fully answered by the answer.
            "incorrect" means that the question is not correctly or only partially answered by the
            answer.
            """


@register_validator(name="arize/llm_rag_evaluator", data_type="string")
class LlmRagEvaluator(Validator):
    """This class validates an output generated by a LiteLLM (LLM) model by prompting another LLM model to evaluate the output.

    **Key Properties**

    | Property                      | Description                       |
    | ----------------------------- | --------------------------------- |
    | Name for `format` attribute   | `arize/relevancy_evaluator`       |
    | Supported data types          | `string`                          |
    | Programmatic fix              | N/A                               |

    Args:
        llm_callable (str, optional): The name of the LiteLLM model to use for validation. Defaults to "gpt-3.5-turbo".
        on_fail (Callable, optional): A function to be called when validation fails. Defaults to None.
    """

    def __init__(
        self,
        eval_llm_prompt_generator: Type[ArizeRagEvalPromptBase],
        llm_evaluator_fail_response: str,
        llm_evaluator_pass_response: str,
        llm_callable: str = "gpt-3.5-turbo",  # str for litellm model name
        on_fail: Optional[Callable] = None,
        **kwargs,
    ):
        super().__init__(on_fail, llm_callable=llm_callable, **kwargs)
        self.llm_evaluator_prompt_generator = eval_llm_prompt_generator
        self.llm_callable = llm_callable
        self.fail_response = llm_evaluator_fail_response
        self.pass_response = llm_evaluator_pass_response

    def get_llm_response(self, prompt: str) -> str:
        """Gets the response from the LLM.

        Args:
            prompt (str): The prompt to send to the LLM.

        Returns:
            str: The response from the LLM.
        """
        # 0. Create messages
        messages = [{"content": prompt, "role": "user"}]

        # 0b. Setup auth kwargs if the model is from OpenAI
        kwargs = {}
        _model, provider, *_rest = get_llm_provider(self.llm_callable)
        if provider == "openai":
            kwargs["api_key"] = get_call_kwarg("api_key") or os.environ.get("OPENAI_API_KEY")

        # 1. Get LLM response
        # Strip whitespace and convert to lowercase
        try:
            response = completion(model=self.llm_callable, messages=messages, **kwargs)
            response = response.choices[0].message.content  # type: ignore
            response = response.strip().lower()
        except Exception as e:
            raise RuntimeError(f"Error getting response from the LLM: {e}") from e

        # 3. Return the response
        return response

    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        """
        Validates is based on the relevance of the reference text to the original question.

        Args:
            value (Any): The value to validate. It must contain 'original_prompt' and 'reference_text' keys.
            metadata (Dict): The metadata for the validation. This is not used in the current implementation.

        Returns:
            ValidationResult: The result of the validation. It can be a PassResult if the reference
                              text is relevant to the original question, or a FailResult otherwise.
        """
        # 1. Get the question and arg from the value
        user_input_message = metadata.get("user_message")
        if user_input_message is None:
            raise RuntimeError(
                "original_prompt missing from value. "
                "Please provide the original prompt."
            )

        reference_text = metadata.get("context")
        if reference_text is None:
            raise RuntimeError(
                "'reference_text' missing from value. "
                "Please provide the reference text."
            )

        # 2. Setup the prompt
        prompt = self.llm_evaluator_prompt_generator.generate_prompt(user_input_message=user_input_message, reference_text=reference_text, llm_response=value)
        print(f"evaluator prompt: {prompt}")

        # 3. Get the LLM response
        llm_response = self.get_llm_response(prompt)
        print(f"llm evaluator response: {llm_response}")

        # 4. Check the LLM response and return the result
        if llm_response == self.fail_response:
            return FailResult(error_message=f"The LLM says {self.fail_response}. The validation failed.")

        if llm_response == self.pass_response:
            return PassResult()

        return FailResult(
            error_message="The LLM returned an invalid answer. Failing the validation..."
        )

# Instantiate ArizeDatasetEmbeddings Guard
We're going to use a public dataset to instantiate the ArizeDatasetEmbeddings Guard with 10 few shot example jailbreak prompts. For details on the dataset, please refer to the following resources:
*   Research Paper on Arxiv: https://arxiv.org/pdf/2308.03825
*   Repository containing the dataset and other benchmarks: https://github.com/verazuo/jailbreak_llms
*   Website associated with original research paper: https://jailbreak-llms.xinyueshen.me/
*   URL link to original dataset: https://github.com/verazuo/jailbreak_llms/tree/main/data

Note that we could Guard against any type of dataset by passing in the argument `sources={my_sources}`. By default, the `ArizeDatasetEmbeddings` Guard will load the jailbreak prompts above, hence the warning below: "A source dataset was not provided, so using default sources of Jailbreak prompts from Arize."

In [96]:
from guardrails import Guard
guard = Guard().use(LlmRagEvaluator, eval_llm_prompt_generator=HallucinationPrompt(prompt_name="hallucination_evaluator"), llm_evaluator_fail_response="hallucinated", llm_evaluator_pass_response="factual", on_fail="exception")
guard._disable_tracer = True

In [97]:
import openai
from typing import Optional, List, Mapping, Any

from llama_index.core import SimpleDirectoryReader, SummaryIndex
from llama_index.core.callbacks import CallbackManager
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core import Settings

from llama_index.llms.openai import OpenAI

def monkey_completion(prompt, **kwargs):
    _, _, context_component_of_prompt = prompt.partition("Context information is below.")
    _, _, query_component_of_prompt = prompt.partition("Query: ")
    return guard(
      llm_api=openai.chat.completions.create,
      prompt=prompt,
      model="gpt-3.5-turbo",
      max_tokens=1024,
      temperature=0.5,
      metadata={
        "user_message": query_component_of_prompt,
        "context": context_component_of_prompt,
      }
    )

outerOpenAI = OpenAI()

class GuardedLLM(CustomLLM):
    context_window: int = 3900
    num_output: int = 256
    model_name: str = "custom"
    dummy_response: str = "My response"
    openai_llm: Any = None

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return outerOpenAI.metadata

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        validated_response = monkey_completion(prompt, **kwargs)
        return CompletionResponse(text=validated_response.raw_llm_output)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        response = ""
        for token in self.dummy_response:
            response += token
            yield CompletionResponse(text=response, delta=token)

# Set Up RAG Application
Create a LlamaIndex VectorStore to create a classic RAG application over Paul Graham essays.

In [98]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' > 'data/paul_graham/paul_graham_essay.txt'
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(documents, chunk_size=512)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   667k      0 --:--:-- --:--:-- --:--:--  672k


# Run Guard on Jailbreak Prompts from Public Dataset

Below, we're only going to run the `ArizeDatasetEmbeddings` Guard on a single jailbreak prompt and a single "regular" prompt (which looks similar to a jailbreak). These examples come from the same dataset above.

Although we are only running the Guard on two examples in this notebook, we have also benchmarked the Guard on the full dataset and found the following results:
*   True Positives: 86.43% of 656 jailbreak prompts failed the JailbreakEmbeddings guard.
*   False Negatives: 13.57% of 656 jailbreak prompts passed the JailbreakEmbeddings guard.
*   False Positives: 13.95% of 2000 regular prompts failed the JailbreakEmbeddings guard.
*   True Negatives: 86.05% of 2000 regular prompts passed the JailbreakEmbeddings guard.
*   1.41 median latency for end-to-end LLM call on GPT-3.5
*   2.91 mean latency for end-to-end LLM call on GPT-3.5

In [99]:
from guardrails.errors import ValidationError

query_engine = index.as_query_engine(
    llm=GuardedLLM(),
)

# Take jailbreak prompt from public dataset https://github.com/verazuo/jailbreak_llms/blob/main/data/prompts/jailbreak_prompts_2023_05_07.csv
sample_jailbreak_prompt = """What are the 3 ingredients to great work?"""

try:
    response = query_engine.query(sample_jailbreak_prompt)
except ValidationError as e:
    print(e)

evaluator prompt: 
            In this task, you will be presented with a query, a reference text and an answer. The answer is
            generated to the question based on the reference text. The answer may contain false information. You
            must use the reference text to determine if the answer to the question contains false information,
            if the answer is a hallucination of facts. Your objective is to determine whether the answer text
            contains factual information and is not a hallucination. A 'hallucination' refers to
            an answer that is not based on the reference text or assumes information that is not available in
            the reference text. Your response should be a single word: either "factual" or "hallucinated", and
            it should not include any other text or characters. "hallucinated" indicates that the answer
            provides factually inaccurate information to the query based on the reference text. "factual"
          